## @classmethod / @staticmethod

Давайте напишем класс `Car`, конструктор которого принимает на вход возраст авто и производителя. Далее определите два метода, первый `from_year` позволит запустить конструктор, если вместо возраста передать год изготовления, а второй `from_str` позволит запустить конструктор, если в него передать строку, содержащую аргументы. Причем в такой строке возраст может идти как до изготовителя, так и после.

In [ ]:
from datetime import date

In [ ]:
date.today().year

2022

In [ ]:
class Car:

    name = 'car'
  
    def __init__(self, age: int, manufacturer: str):
      self.age = age
      self.manufacturer = manufacturer

    @classmethod
    def from_year(cls, year: int, manufacturer: str):
      cls.name = 'not car'
      return cls(date.today().year - year, manufacturer)

    @classmethod 
    def from_str(cls, text: str):
      print()
      if text.split()[0].isdigit():
        return cls(int(text.split()[0]), text.split()[1])
      else:
        return cls(int(text.split()[1]), text.split()[0])
    
    @staticmethod
    def isImport(manufacturer:  str):
      return manufacturer in {'BMW', 'Wolvo', 'Mercedes'}

In [ ]:
Car.name

'car'

In [ ]:
c = Car.from_year(2014, 'Lada')
print(c.age, c.manufacturer)

8 Lada


In [ ]:
Car.name

'not car'

In [ ]:
d = Car.from_str('Lada 2010')
print(d.age, d.manufacturer)


2010 Lada


In [ ]:
d = Car.from_str('2010 KaMaz')
print(d.age, d.manufacturer)


2010 KaMaz


In [ ]:
d.isImport(d.manufacturer)

NameError: ignored

In [ ]:
Car.isImport('KaMaZ')

NameError: ignored

## @property

Давайте напишем родительский класс `Vehicle`, от которого будет наследоваться класс `Car`. У класса `Vehicle` должны быть атрибуты `power`, `cost`, `weight` и `name`. У класса `Car` должен быть дополнительный атрибут `color`, а также `speed`, которое вычисляется как $\frac{power}{weight} * 100$.

In [ ]:
class Vehicle:
  
  def __init__(self, power: float, cost: float, weight: float, name: str):
    self.power = power
    self.cost = cost
    self.weight = weight
    self.name = name

  @property
  def power(self):
    print('Возвращаю значение')
    return self.__power

  @power.setter
  def power(self, value: float):
    print('Присваиваю значение')
    if value < 0:
      raise ValueError('Negative power')
    self.__power = value

class Car(Vehicle):

  def __init__(self, power: float, cost: float, weight: float, name: str, color: str):
    super().__init__(power, cost, weight, name)
    self.color = color
    self.speed = self.power / self.weight * 100


In [ ]:
v = Vehicle(1000, 10, 100, 'Vehicle')
v.power

Присваиваю значение
Возвращаю значение


1000

In [ ]:
c = Car(1000, 10, 100, 'Car', 'red')
print(c.color, c.speed)

Присваиваю значение
Возвращаю значение
red 1000.0


## Динамические атрибуты

In [ ]:
class AttrDict(dict):
  
  def __init__(self, d: dict):
    for key, value in d.items():
      self[key] = value

  def __getattr__(self, key):
    return self[key]

In [ ]:
d = {'aaaa': 1, 'dfgdf': {123123: 12312, 132123: 12312}, 'pop_': 123123}
ad = AttrDict(d)
print(ad.pop_)
print(ad.dfgdf)

123123
{123123: 12312, 132123: 12312}


Теперь давайте реализуем возможность создания динамических атрибутов у питоновских объектов. Представьте, что вам на вход подается словарь, в котором значениями могут быть разные типы данных, в том числе и словари. Например, на вход мы получили:

```
      {
        "title": "iPhone X",
        "price": 100,
        "class": "smartphone",
        "location": {
        "address": "город Самара, улица Мориса Тореза, 50",
        "metro_stations": ["Спортивная", "Гагаринская"]
        }
      }
```

И хотим суметь обратиться к `obj.location.address`.

Отдельно подумайте, что нужно делать, если ключ словаря совпадает со встроенными командами питона.

In [ ]:
import keyword
import json

In [ ]:
class JSON2Python:

  def __init__(self, json_obj):
    for key, value in json_obj.items():
      if type(value) == dict:
        JSON2Python.dynamic_attr(key, AttrDict(value))
      else:
        JSON2Python.dynamic_attr(key, value)

  @classmethod
  def dynamic_attr(cls, key, value):
    if keyword.iskeyword(key):
      setattr(cls, str(key) + '_', value)
    else:
      setattr(cls, key, value)

In [ ]:
example = """
{
  "title": "iPhone X",
  "price": 100,
  "class": "smartphone",
  "location": 
    {
      "address": "город Самара, улица Мориса Тореза, 50",
      "metro_stations": ["Спортивная", "Гагаринская"]
    },
  "if": "pop"
}
"""

In [ ]:
jp = JSON2Python(json.loads(example))

In [ ]:
jp.if_

'pop'

**Задача**: написать класс DivStr, который будет наследоваться от класса str, а также будет поддерживать операции `//` и `%`.

In [ ]:
class DivStr(str):
  
  def __init__(self, *args, **kwargs):
    super().__init__()

  def __floordiv__(self, k: int):
    if k <= 0:
      raise ValueError('Nonpositive divisor')
    d = len(self) // k
    if d > 0:
      return ' '.join(self[i * d: (i + 1) * d] for i in range(0, k))
    return ''

  def __mod__(self, k: int):
    if len(self) % k == 0:
      return ''
    return self[-(len(self) % k):]

In [ ]:
ds = DivStr('aaaaaaaaaaaaaaa')
print(ds, len(ds), ds.strip())

aaaaaaaaaaaaaaa 15 aaaaaaaaaaaaaaa


In [ ]:
print(ds // 17) # ''
print(ds // 5) # 'aaa aaa ...'
print(ds // 15) #


aaa aaa aaa aaa aaa
a a a a a a a a a a a a a a a


In [ ]:
ds // 7

'aa aa aa aa aa aa aa'

In [ ]:
print(ds // 17, ds % 17)
print(ds // 7, ds % 7)
print(ds // 5, ds % 5)
print(ds // 2, ds % 2)

 aaaaaaaaaaaaaaa
aa aa aa aa aa aa aa a
aaa aaa aaa aaa aaa 
aaaaaaa aaaaaaa a


In [ ]:
s = 'a' * 15
d = len(s) // 7
for i in range(0, 7):
  print(i * d, (i + 1) * d, s[i * d: (i + 1) * d])

0 2 aa
2 4 aa
4 6 aa
6 8 aa
8 10 aa
10 12 aa
12 14 aa
